In [2]:
d = '../data/pre/' # raw data directory
print('dev stat 22666633')
import os
import pandas as pd
import numpy as np
from time import time
from sklearn.metrics import log_loss

from keras.models import load_model, Sequential, Model
from keras.utils import plot_model, to_categorical
from keras.callbacks import TensorBoard, ModelCheckpoint, Callback
from keras.layers import Dense, Embedding, LSTM, GRU, SimpleRNN, BatchNormalization
from keras.layers import Dropout, Bidirectional, Flatten, Input, Reshape
from keras.layers.merge import Concatenate, Add, concatenate, add
from keras.optimizers import rmsprop, sgd

ac = pd.read_csv('%sapp_categories.csv' % d)
ui = pd.read_csv('%suser_installedapps.csv' % d)
ua = pd.read_csv('%suser_app_actions.csv' % d)

tr_ori = pd.read_csv('../data/pre/new_generated_train.csv', index_col=0)
te_ori = pd.read_csv('../data/pre/new_generated_test.csv', index_col=0)
# features = [ 'positionType', 'connectionType', 'age', 'haveBaby', 'telecomsOperator',
#             'gender', 'education', 'clickTime_h', 'clickTime_d', 'weekDay',
#             'marriageStatus', 'appPlatform', 'clickTime_m']
'''OrderedDict([('appCategory', 0.11342039643011995),
           ('positionID', 0.097800108293958354),
           ('positionType', 0.087460470534174911),
           ('creativeID', 0.071309827665513831),
           ('appID', 0.06958037043067912),
           ('adID', 0.063169867995414214),
           ('advertiserID', 0.05492409862736055),
           ('camgaignID', 0.050796291439061121),
           ('sitesetID', 0.013700972348893198),
           ('connectionType', 0.010308483261562163),
           ('residence', 0.0075065408905755091),
           ('age', 0.006662250035074235),
           ('hometown', 0.0057415423374340075),
           ('haveBaby', 0.0050835138043705281),
           ('telecomsOperator', 0.0049696062876111837),
           ('gender', 0.0046607920728349225),
           ('education', 0.0030160253068509456),
           ('clickTime_h', 0.0029165451700999038),
           ('clickTime_d', 0.0028077837477102278),
           ('weekDay', 0.001772181032721335),
           ('marriageStatus', 0.0016869540469555094),
           ('appPlatform', 0.0008981783007564663),
           ('clickTime_m', 0.00072661907607473827),
           ('userID', unkown)])'''

dev stat 22666633


Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


"OrderedDict([('appCategory', 0.11342039643011995),\n           ('positionID', 0.097800108293958354),\n           ('positionType', 0.087460470534174911),\n           ('creativeID', 0.071309827665513831),\n           ('appID', 0.06958037043067912),\n           ('adID', 0.063169867995414214),\n           ('advertiserID', 0.05492409862736055),\n           ('camgaignID', 0.050796291439061121),\n           ('sitesetID', 0.013700972348893198),\n           ('connectionType', 0.010308483261562163),\n           ('residence', 0.0075065408905755091),\n           ('age', 0.006662250035074235),\n           ('hometown', 0.0057415423374340075),\n           ('haveBaby', 0.0050835138043705281),\n           ('telecomsOperator', 0.0049696062876111837),\n           ('gender', 0.0046607920728349225),\n           ('education', 0.0030160253068509456),\n           ('clickTime_h', 0.0029165451700999038),\n           ('clickTime_d', 0.0028077837477102278),\n           ('weekDay', 0.001772181032721335),\n       

In [3]:
f = 'positionID'
va = tr_ori.sample(frac=0.1, random_state=62)
tr = tr_ori.drop(va.index, axis=0)

va_y = va.label.values.reshape(-1,1)
tr_y = tr.label.values.reshape(-1,1)
va_x = va[f].values.reshape(-1,1)
tr_x = tr[f].values.reshape(-1,1)

va_ = va.groupby(f).apply(lambda df: np.mean(df.label))
tr_ = tr.groupby(f).apply(lambda df: np.mean(df.label))
te_ = te_ori.groupby(f).apply(lambda df: len(df.label))


tr_stat = tr.groupby(f).apply(lambda df: len(df))

print(tr_stat.values)

[  1  10 129 ..., 111   1   2]


In [ ]:
total_tr = 2*max(tr_stat.values)
new_x = []
new_y = []
for cate,occ in zip(tr_stat.index, tr_stat.values):
    tmp = tr.loc[tr[f]==cate]
    x = tmp[f].values
    y = tmp.label.values
    x = list(x)
    y = list(y)
    x = x*(int(total_tr/len(x)))
    y = y*(int(total_tr/len(y))) 
    print(len(x))   
    new_x += x
    new_y += y
    del x,y, tmp
new_x = np.array(new_x).reshape(-1,1)
new_y = np.array(new_y).reshape(-1,1)
print('Length of new x:', len(new_x))

In [10]:
tr_, tr_stat, te_

(positionID
 1       0.000000
 2       0.200000
 3       0.015504
 5       0.000000
 6       0.000000
 7       0.000000
 8       0.000000
 9       0.000000
 10      0.017857
 12      0.000000
 13      0.000000
 14      0.000000
 15      0.113636
 16      0.000000
 17      0.000000
 18      0.000000
 19      0.007207
 20      0.333333
 22      0.000000
 23      0.500000
 24      0.000000
 25      0.006364
 26      0.000000
 27      0.000000
 28      0.125000
 29      0.014925
 31      0.000000
 32      0.000000
 33      0.000000
 34      0.020000
           ...   
 7614    0.000000
 7615    0.079208
 7616    0.054054
 7617    0.000000
 7618    0.000000
 7619    0.016959
 7620    0.000000
 7621    0.080000
 7622    0.111111
 7623    0.000000
 7624    0.065217
 7625    0.000000
 7627    0.000000
 7628    0.000000
 7629    0.000000
 7630    0.250000
 7631    0.250000
 7632    0.000000
 7633    0.031609
 7634    0.000000
 7635    0.000000
 7636    0.000000
 7637    0.001980
 7638    0.00591

In [5]:
# no ajust
np.random.seed(323)
i = Input(shape=(1,))
o = Embedding(np.max(tr_x)+1, 16)(i)
o = Flatten()(o)
o = Dense(64, activation='tanh')(o)
o = Dense(1, activation='sigmoid')(o)
model_ = Model(i,o)
# model_.summary()
model_.compile(optimizer=rmsprop(lr=.0001), loss='binary_crossentropy', metrics=['binary_crossentropy'])

In [11]:
model_.fit(tr_x, tr_y,validation_data=(va_x,va_y), verbose=1, epochs=20, batch_size=1024,  shuffle=True)
print('\nUnbalanced model predict:\n', model_.predict(te_.index))

Train on 3374575 samples, validate on 374953 samples
Epoch 1/20
3374575/3374575 [==============================] - 8s - loss: 0.1088 - binary_crossentropy: 0.1088 - val_loss: 0.1102 - val_binary_crossentropy: 0.1102
Epoch 2/20
3374575/3374575 [==============================] - 9s - loss: 0.1088 - binary_crossentropy: 0.1088 - val_loss: 0.1102 - val_binary_crossentropy: 0.1102
Epoch 3/20
3374575/3374575 [==============================] - 9s - loss: 0.1088 - binary_crossentropy: 0.1088 - val_loss: 0.1101 - val_binary_crossentropy: 0.1101
Epoch 4/20
3374575/3374575 [==============================] - 9s - loss: 0.1088 - binary_crossentropy: 0.1088 - val_loss: 0.1101 - val_binary_crossentropy: 0.1101
Epoch 5/20
3374575/3374575 [==============================] - 9s - loss: 0.1088 - binary_crossentropy: 0.1088 - val_loss: 0.1101 - val_binary_crossentropy: 0.1101
Epoch 6/20
3374575/3374575 [==============================] - 9s - loss: 0.1088 - binary_crossentropy: 0.1088 - val_loss: 0.1101 - v

In [19]:
# balanced
new_x = tr_x
new_y = tr_y
np.random.seed(323)
i = Input(shape=(1,))
o = Embedding(np.max(new_x)+1, 16)(i)
o = Flatten()(o)
o = Dense(64, activation='tanh')(o)
o = Dense(1, activation='sigmoid')(o)
model = Model(i,o)
# model.summary()
model.compile(optimizer=rmsprop(lr=.0001), loss='binary_crossentropy', metrics=['binary_crossentropy'])

In [73]:
model.fit(new_x, new_y,validation_data=(va_x,va_y), verbose=1, epochs=2, batch_size=4096,  shuffle=True)

print('\nBalanced model predict:\n', model.predict(te_.index))


Train on 47197066 samples, validate on 374953 samples
Epoch 1/2
47197066/47197066 [==============================] - 51s - loss: 0.1980 - binary_crossentropy: 0.1980 - val_loss: 0.1133 - val_binary_crossentropy: 0.1133
Epoch 2/2
47197066/47197066 [==============================] - 50s - loss: 0.1980 - binary_crossentropy: 0.1980 - val_loss: 0.1133 - val_binary_crossentropy: 0.1133

Balanced model predict:
 [[ 0.01565103]
 [ 0.02915289]
 [ 0.03406202]
 [ 0.00715058]
 [ 0.03182879]
 [ 0.0204984 ]
 [ 0.02702323]
 [ 0.03722728]
 [ 0.20242479]
 [ 0.03080541]]
ideal-model ideal-model_ model_-model
-0.00001040, 0.00002878, -0.00003918
appCategory
0      0.158049
2      0.000490
101    0.003711
104   -0.000479
106    0.039852
108    0.000264
201   -0.000524
203   -0.000934
209   -0.000027
301    0.003242
402    0.000538
407   -0.004852
408   -0.051205
503    0.000976
dtype: float64 
Balanced model
 appCategory
0      6.232816e-03
2     -2.146239e-04
101    8.914894e-04
104    7.539287e-04
106 

In [21]:
tr__df = tr_.to_frame().reset_index()
dict_prob = tr__df.set_index(tr__df[f]).to_dict()[0]
for c in va_x:
    if c[0] not in dict_prob: dict_prob[c[0]] = np.mean(tr_.values)
ideal_loss = log_loss(va_y, np.array([dict_prob[c[0]] for c in va_x]).reshape(-1,1))
model_loss = log_loss(va_y, model.predict(va_x ))
model_loss_ = log_loss(va_y, model_.predict(va_x))
print('ideal-model', 'ideal-model_', 'model_-model')
print('%.8f, %.8f, %.8f'%(ideal_loss-model_loss , model_loss_-model_loss,ideal_loss-model_loss_))

# print(tr_-np.ravel(model_.predict(va_.index)), '\nBalanced model\n',tr_-np.ravel(model.predict(va_.index)) )

# for test data
te__df = te_.to_frame().reset_index()
te_dict = te__df.set_index(te__df[f]).to_dict()[0]
new_loss_df = pd.merge(te_ori, tr__df, on=f, how='left')
new_loss_df[1] = model.predict(te_ori[f])
new_loss_df[2] = model_.predict(te_ori[f])
# ======================== #
loss_test = np.mean(new_loss_df[0].values-new_loss_df[1].values)
loss_test_ = np.mean(new_loss_df[0].values-new_loss_df[2].values)
loss_test, loss_test_

ideal-model ideal-model_ model_-model
-0.57558677, -0.57881036, 0.00322359


(nan, nan)

In [80]:
model.save('balanced_tl/%s_%.8f.h5'%(f,loss_test))

In [23]:
new_loss_df[0].values-new_loss_df[2].values

array([ 0.00043276, -0.00367733, -0.00050406, ..., -0.00106709,
        0.00917387, -0.00114787])

In [25]:
loss_test_ = np.mean(new_loss_df[0].values-new_loss_df[2].values)
loss_test_

nan

In [3]:
d = '../data/pre/' # raw data directory
print('dev stat 22666633')
import os
import pandas as pd
import numpy as np
from time import time
import math
from sklearn.metrics import log_loss

def i_log(y):
  if y<= 0: y = 1e-5
  if y>=1: y = 1 - 1e-5
  return math.log(y/(1-y))


features = ['appCategory', 'positionID', 'positionType', 'creativeID', 'appID', 'adID',
            'advertiserID', 'camgaignID', 'sitesetID', 'connectionType',
            'residence', 'age', 'hometown', 'haveBaby', 'telecomsOperator',
            'gender', 'education', 'clickTime_h', 'clickTime_d', 'weekDay',
            'marriageStatus', 'appPlatform', 'clickTime_m', 'userID']

tr_ori = pd.read_csv('../data/pre/new_generated_train.csv', index_col=0)
te_ori = pd.read_csv('../data/pre/new_generated_test.csv', index_col=0)

maxs = pd.concat([tr_ori[features], te_ori[features]]).max()+1
# features = [ 'positionType', 'connectionType', 'age', 'haveBaby', 'telecomsOperator',
#             'gender', 'education', 'clickTime_h', 'clickTime_d', 'weekDay',
#             'marriageStatus', 'appPlatform', 'clickTime_m']
'''OrderedDict([('appCategory', 0.11342039643011995),
           ('positionID', 0.097800108293958354),
           ('positionType', 0.087460470534174911),
           ('creativeID', 0.071309827665513831),
           ('appID', 0.06958037043067912),
           ('adID', 0.063169867995414214),
           ('advertiserID', 0.05492409862736055),
           ('camgaignID', 0.050796291439061121),
           ('sitesetID', 0.013700972348893198),
           ('connectionType', 0.010308483261562163),
           ('residence', 0.0075065408905755091),
           ('age', 0.006662250035074235),
           ('hometown', 0.0057415423374340075),
           ('haveBaby', 0.0050835138043705281),
           ('telecomsOperator', 0.0049696062876111837),
           ('gender', 0.0046607920728349225),
           ('education', 0.0030160253068509456),
           ('clickTime_h', 0.0029165451700999038),
           ('clickTime_d', 0.0028077837477102278),
           ('weekDay', 0.001772181032721335),
           ('marriageStatus', 0.0016869540469555094),
           ('appPlatform', 0.0008981783007564663),
           ('clickTime_m', 0.00072661907607473827),
           ('userID', unkown)])'''






dev stat 22666633


/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


"OrderedDict([('appCategory', 0.11342039643011995),\n           ('positionID', 0.097800108293958354),\n           ('positionType', 0.087460470534174911),\n           ('creativeID', 0.071309827665513831),\n           ('appID', 0.06958037043067912),\n           ('adID', 0.063169867995414214),\n           ('advertiserID', 0.05492409862736055),\n           ('camgaignID', 0.050796291439061121),\n           ('sitesetID', 0.013700972348893198),\n           ('connectionType', 0.010308483261562163),\n           ('residence', 0.0075065408905755091),\n           ('age', 0.006662250035074235),\n           ('hometown', 0.0057415423374340075),\n           ('haveBaby', 0.0050835138043705281),\n           ('telecomsOperator', 0.0049696062876111837),\n           ('gender', 0.0046607920728349225),\n           ('education', 0.0030160253068509456),\n           ('clickTime_h', 0.0029165451700999038),\n           ('clickTime_d', 0.0028077837477102278),\n           ('weekDay', 0.001772181032721335),\n       

In [6]:
f = 'appCategory'

tr_ = tr_ori.groupby(f).apply(lambda df: np.mean(df.label))

tr__df = tr_.to_frame().reset_index()
tr_dict = tr__df.set_index(tr__df[f]).to_dict()[0]

In [7]:
tr_dict

{0: 0.42384105960264901,
 2: 0.015486711799687925,
 101: 0.030474040632054177,
 104: 0.034927744522118505,
 106: 0.044642857142857144,
 108: 0.0069377401707460202,
 201: 0.031497666957368657,
 203: 0.09658498435870698,
 209: 0.020328219191233017,
 301: 0.028041898153231295,
 402: 0.037568127519784979,
 407: 0.20372257523078768,
 408: 0.0,
 503: 0.031987423465166309}